In [1]:

from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import tensorflow

import torch.nn as nn
import torch.nn.functional as F
import pickle
import torchvision
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
#Reading the distileld images from a pickle file
with open('image_data.pkl','rb') as f: data = pickle.load(f)

In [3]:
d = []
for i in data:
    d.append({
        "round": i[0],
        "label": i[1],
        "img": i[2]
             })

In [4]:
import pandas as pd
df = pd.DataFrame(d)

In [5]:
df

,round,label,img
0,0,0,"[[[-0.7581059], [0.063398585], [0.3590322], [-..."
1,0,1,"[[[0.840746], [0.6316065], [-1.2231346], [-0.9..."
2,0,2,"[[[-1.7556419], [2.9030576], [0.96203077], [0...."
3,0,3,"[[[-1.3797349], [0.24783106], [-1.3950331], [-..."
4,0,4,"[[[-0.40310335], [1.279251], [0.42541805], [-0..."
...,...,...,...
295,29,5,"[[[0.97014874], [-0.62902826], [-0.94049585], ..."
296,29,6,"[[[-1.7039658], [0.7368676], [-1.3237106], [-0..."
297,29,7,"[[[0.5628856], [-2.053446], [-0.33936182], [0...."
298,29,8,"[[[-0.6217524], [-2.370984], [0.662596], [0.75..."


In [6]:
# Selecting the last round images as our training images
osi = df[df["round"] == 29] #one shot images

In [7]:
osi

,round,label,img
290,29,0,"[[[-1.1906161], [-1.607573], [1.5480994], [-1...."
291,29,1,"[[[0.43251985], [0.5482655], [-1.0812683], [-0..."
292,29,2,"[[[-0.45173714], [-1.7306507], [-1.1495376], [..."
293,29,3,"[[[-1.1158541], [-0.5300463], [-0.7418438], [1..."
294,29,4,"[[[-0.2796815], [0.19280227], [-0.01091018], [..."
295,29,5,"[[[0.97014874], [-0.62902826], [-0.94049585], ..."
296,29,6,"[[[-1.7039658], [0.7368676], [-1.3237106], [-0..."
297,29,7,"[[[0.5628856], [-2.053446], [-0.33936182], [0...."
298,29,8,"[[[-0.6217524], [-2.370984], [0.662596], [0.75..."
299,29,9,"[[[-1.14523], [1.3418516], [-1.1072803], [-0.2..."


In [8]:
# osi['img'].iloc[0]

In [9]:
from PIL import Image
from matplotlib import cm
img = np.uint8(osi['img'].iloc[0])
img = img[..., 0]
img = Image.fromarray(img)

img.save('test.png')

In [10]:
img.show()

In [11]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

y_train = list(map(int, list(osi["label"])))
# Scale images to the [0, 1] range
x_train = list(map(lambda x: x.astype("float32") / 255, list(osi["img"])))
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.array(x_train)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (10, 28, 28, 1)
10 train samples
10000 test samples


In [53]:
model = Sequential()
model.add(keras.Input(shape=input_shape))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 10, 10, 48)        38448     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 48)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               307456    
_________________________________________________________________
dense_19 (Dense)             (None, 84)               

In [49]:
batch_size = 10
epochs = 50

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/50
1/1 [==============================] - 1s 843ms/step - loss: 2.3026 - accuracy: 0.1111 - val_loss: 2.3137 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 58ms/step - loss: 2.3010 - accuracy: 0.1111 - val_loss: 2.3375 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 57ms/step - loss: 2.2978 - accuracy: 0.2222 - val_loss: 2.3787 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 65ms/step - loss: 2.2929 - accuracy: 0.1111 - val_loss: 2.4430 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 65ms/step - loss: 2.2861 - accuracy: 0.2222 - val_loss: 2.5457 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 54ms/step - loss: 2.2768 - accuracy: 0.1111 - val_loss: 2.7108 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 61ms/step - loss: 2.2646 - accuracy: 0.1111 - val_loss: 2.9700 - val_accuracy: 0.0000e+0

In [50]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 44.50741195678711
Test accuracy: 0.46630001068115234


In [58]:
def run(n):
    sum_a = 0
    sum_l = 0
    for i in range(n):
        model = Sequential()
        model.add(keras.Input(shape=input_shape))
        model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
        model.add(MaxPool2D(strides=2))
        model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
        model.add(MaxPool2D(strides=2))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(84, activation='relu'))
        model.add(Dense(10, activation='softmax'))

    #     model.summary()

        batch_size = 10
        epochs = 50

        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
        
        score = model.evaluate(x_test, y_test, verbose=0)
        print("Test loss:", score[0])
        print("Test accuracy:", score[1])
        
        sum_a = sum_a+score[1]
        sum_l = sum_l+score[0]
    avg_a = sum_a/n
    avg_l = sum_l/n
    return (avg_a, avg_l)

In [57]:
avg_a, avg_l = run(20)
print("Test loss:", avg_l)
print("Test accuracy:", avg_a)

Epoch 1/50
1/1 [==============================] - 1s 717ms/step - loss: 2.3025 - accuracy: 0.0000e+00 - val_loss: 2.3167 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 74ms/step - loss: 2.3003 - accuracy: 0.1111 - val_loss: 2.3376 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 75ms/step - loss: 2.2973 - accuracy: 0.1111 - val_loss: 2.3764 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 132ms/step - loss: 2.2928 - accuracy: 0.1111 - val_loss: 2.4372 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 108ms/step - loss: 2.2864 - accuracy: 0.2222 - val_loss: 2.5318 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 99ms/step - loss: 2.2774 - accuracy: 0.1111 - val_loss: 2.6824 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 102ms/step - loss: 2.2650 - accuracy: 0.1111 - val_loss: 2.9156 - val_accuracy: 0.

In [59]:
avg_a, avg_l = run(50)
print("AVG Test Loss:", avg_l)
print("AVG Test Accuracy:", avg_a)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 2.3027 - accuracy: 0.2222 - val_loss: 2.3090 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 75ms/step - loss: 2.3014 - accuracy: 0.1111 - val_loss: 2.3225 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 75ms/step - loss: 2.2992 - accuracy: 0.1111 - val_loss: 2.3437 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 78ms/step - loss: 2.2964 - accuracy: 0.1111 - val_loss: 2.3770 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 77ms/step - loss: 2.2924 - accuracy: 0.2222 - val_loss: 2.4271 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 75ms/step - loss: 2.2874 - accuracy: 0.1111 - val_loss: 2.5010 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 75ms/step - loss: 2.2807 - accuracy: 0.1111 - val_loss: 2.6127 - val_accuracy: 0.0000e+00
E